In [1]:
import os
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv
import json


In [2]:
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

MODEL='gpt-4o-mini'
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [6]:
ticket_prices = {"london":"$599","berlin":"$799","rejkjavik":"$5999","bengaluru":"$599","sfo":"$399"}

def get_ticket_price(destination_city):
    print(f"Get ticket price called for the city: {destination_city}")
    return ticket_prices.get(destination_city.lower(), "Unknown")

In [7]:
bookings = {"london":1,"berlin":2,"rejkjavik":3,"bengaluru":6,"sfo":9}

def book(destination_city, num):
    print(f"Request for booking {num} tickets for the city: {destination_city}")
    if destination_city.lower() in bookings.keys():
        tmp_bookings = bookings.get(destination_city.lower()) + num
        bookings[destination_city.lower()] = tmp_bookings
        return True
    else:
        print(f"Booking not possible for {destination_city}")
        return False

In [8]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [14]:
booking_function = {
    "name": "book",
    "description": "When the customer wants to book one or more tickets for a destination city, call this function to store a booking. For example when a customer requests 'please book me 2 ticktets to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to book tickets for",
            },
            "num": {
                "type":"integer",
                "description": "Number of tickets the customer wants to book for the given destination city",
            },
        },
        "required": ["destination_city", "num"],
        "additionalProperties": False
    }
}

In [15]:
tools = [{"type": "function", "function": price_function},
         {"type": "function", "function": booking_function}
        ]

In [16]:
def handle_tool_call(message):
    print(f"Got a handle_tool_call: {message}")
    
    tool_call = message.tool_calls[0]
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    print(f"Tool call object: {tool_call}")
    print(f"Function called: {function_name}")
    
    if function_name == "get_ticket_price":
        price = get_ticket_price(city)
        response_content = {"destination_city": city, "price": price}
        print(f"Got price for the {city} as {price}")

    elif function_name == "book":
        num = arguments.get('num')
        booking_success = book(city, num)
        response_content = {"destination_city": city, "num": num, "success": booking_success}

    else:
        response_content = {"error": f"Unknown function: {function_name}"}

    response = {
        "role":"tool",
        "content" : json.dumps(response_content),
        "tool_call_id":tool_call.id        
    }
    return response, city

In [17]:
import base64
from PIL import Image
from io import BytesIO

In [18]:
def artist(city):

    message = f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style"
    
    response = openai.images.generate(
        model="dall-e-2",
        prompt=message,
        size="1024x1024",
        n=1,
        response_format="b64_json"                
    )
    image_base64 = response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
image = artist("New York City")
display(image)

In [54]:
from IPython.display import Audio, display

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

talker("well, it's sunny out there and I am wondering if I should go out and do some sports activity")

In [19]:
def chat(history):
    messages = [{"role":"system", "content":system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        #image = artist(city)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    #talker(reply)

    return history, image
        
        

In [43]:
def translate2German(history):

    assistant_message = history[-1]
    assistant_message_content = assistant_message.get("content", "")
    
    system_message = "You are a translator between English and German. You translate text from English to German in a concise, simple manner"
    user_prompt = f"Translate the following text to German: {assistant_message_content}"

    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages = [{"role": "system", "content": system_message},{"role":"user", "content":user_prompt}]
    )
    translated_message = response.choices[0].message.content
    return translated_message
    

In [44]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        translated_output = gr.Textbox(label="Translated Response (German)", interactive=False)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    ).then(translate2German, inputs=chatbot, outputs=translated_output)
    
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7880
* To create a public link, set `share=True` in `launch()`.


Got a handle_tool_call: ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_VnRl3vgcYWPjNdQC4EyYsbsb', function=Function(arguments='{"destination_city":"SFO"}', name='get_ticket_price'), type='function')])
Tool call object: ChatCompletionMessageFunctionToolCall(id='call_VnRl3vgcYWPjNdQC4EyYsbsb', function=Function(arguments='{"destination_city":"SFO"}', name='get_ticket_price'), type='function')
Function called: get_ticket_price
Get ticket price called for the city: SFO
Got price for the SFO as $399
